In [27]:
import json
import os 
from datetime import datetime

#for fp in os.listdir('data/2022-10-03'):
#    print(fp)

fp = "data/dev/9000000183.json"


    

In [54]:
with open(fp) as f:
    data = json.load(f)
df = pd.DataFrame([{'datetime': r['date'], 'bike': r['bike'], 'car': r['car'], 'pedestrian': r['pedestrian']} for r in data['report']])
df.index = pd.to_datetime(df['datetime'], format='%Y-%m-%dT%H:%M:%S.000Z')
del df['datetime']
df = df.resample('D').sum()

In [55]:
df.mean()

bike          55.556306
car           57.286383
pedestrian    33.539490
dtype: float64